In [ ]:
# 1) Modelo de spaCy (una vez por entorno)
import subprocess, sys
try:
    import es_core_news_sm  # noqa
except Exception:
    subprocess.check_call([sys.executable, "-m", "spacy", "download", "es_core_news_sm"])


In [ ]:
import pandas as pd

import re
import unicodedata
import numpy as np
import langid

# Bots
import hashlib
from collections import Counter
from collections import defaultdict
from math import log2
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from pathlib import Path

# Clusterización
import spacy


In [50]:
df = pd.read_csv("../data/processed/7_final_label.csv")

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113583 entries, 0 to 113582
Data columns (total 51 columns):
 #   Column                       Non-Null Count   Dtype              
---  ------                       --------------   -----              
 0   comment_id                   113583 non-null  object             
 1   comment                      113583 non-null  object             
 2   comment_text_length          113583 non-null  int64              
 3   user_id                      113583 non-null  object             
 4   user_name                    113583 non-null  object             
 5   comment_time                 113583 non-null  datetime64[ns, UTC]
 6   comment_likes                113583 non-null  int64              
 7   total_reply_count            113583 non-null  int64              
 8   is_top_level_comment         113583 non-null  bool               
 9   video_title                  113583 non-null  object             
 10  channel_title                113

# Detección del idioma

In [52]:
# ===========================================
# Detectar idioma → df["lang"] (ISO-639-1)
#  - NaN si el texto es muy corto o casi no lingüístico (emojis/URLs)
#  - RU/UK por alfabeto (cirílico vs latino)
#  - langid obligatorio (sin fallback a 'es')
# ===========================================

URL_RX   = re.compile(r"http[s]?://|www\.", re.I)
EMOJI_RX = re.compile(r"[^\w\s,.\-¡!¿?\(\)\"'@#:/]", re.I)    # aproximado
RX_LAT   = re.compile(r"[a-záéíóúñü]", re.I)
RX_CYR   = re.compile(r"[а-яіїєґё]", re.I)

# Opcional (mejora precisión): restringimos a idiomas más probables del corpus
langid.set_languages(["es","ru","uk","en","pt","it","fr","de"])

def _strip_urls(s: str) -> str:
    return URL_RX.sub("", s)

def _language_of(text: str) -> str:
    if text is None: return np.nan
    raw = str(text)
    if not raw.strip(): return np.nan

    lat = len(RX_LAT.findall(raw))
    cyr = len(RX_CYR.findall(raw))
    non_alpha_frac = len(EMOJI_RX.findall(raw)) / max(1, len(raw))

    # NaN si casi no hay letras o es mayormente símbolos/emojis
    if (lat + cyr) < 3 or non_alpha_frac > 0.6:
        return np.nan

    # Cirílico predominante → ru/uk por letras exclusivas
    if cyr > lat and cyr > 3:
        return "uk" if re.search(r"[іїєґ]", raw) else "ru"

    # Clasificación con langid (sobre texto con URLs removidas)
    lg = langid.classify(_strip_urls(raw))[0]
    lg = str(lg).lower()
    return lg if (len(lg) == 2 and lg.isalpha()) else np.nan

df["lang"] = df["comment"].apply(_language_of)

print("df['lang'] creado (ISO-639-1; NaN si indeterminado)")


✅ df['lang'] creado (ISO-639-1; NaN si indeterminado)


# Detección del Pais

In [54]:
# ==========================================================
# País/Región AUTO-IDENTIFICADO por USUARIO (LatAm + Europa)
#   - Lee señales en comentarios (flags 🇦🇷, "saludos desde...", gentilicios, hashtags)
#   - Asigna 2 columnas nuevas en el master:
#       user_country (ISO-2) y user_region (Europe/LatAm/Other)
#   - Sin exports (solo define OUTPUT_DIR para más adelante)
# ==========================================================


OUTPUT_DIR = "../data/processed"  # definido para el final del proyecto, no se usa ahora

def _norm(s: str) -> str:
    s = str(s).lower()
    return unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")

# --- mapas de países (alias + gentilicios) ---
ALIAS = {
    # Europa
    "espana":"ES","españa":"ES","francia":"FR","alemania":"DE","italia":"IT","portugal":"PT","polonia":"PL",
    "suiza":"CH","austria":"AT","suecia":"SE","noruega":"NO","finlandia":"FI","dinamarca":"DK","estonia":"EE","letonia":"LV","lituania":"LT",
    "chequia":"CZ","republica checa":"CZ","eslovaquia":"SK","hungria":"HU","grecia":"GR","bulgaria":"BG","rumania":"RO","romania":"RO",
    "croacia":"HR","serbia":"RS","bosnia":"BA","eslovenia":"SI",
    # LatAm
    "argentina":"AR","mexico":"MX","méxico":"MX","chile":"CL","uruguay":"UY","paraguay":"PY","peru":"PE","perú":"PE",
    "bolivia":"BO","colombia":"CO","ecuador":"EC","venezuela":"VE","costa rica":"CR","panama":"PA","panamá":"PA",
    "guatemala":"GT","honduras":"HN","nicaragua":"NI","el salvador":"SV","republica dominicana":"DO","cuba":"CU","puerto rico":"PR",
}
DEMONYMS = {
    # Europa
    "espanol":"ES","español":"ES","espanola":"ES","española":"ES","frances":"FR","francesa":"FR","aleman":"DE","alemana":"DE",
    "italiano":"IT","italiana":"IT","portugues":"PT","portuguesa":"PT","polaco":"PL","polaca":"PL",
    "britanico":"GB","británica":"GB","irlandes":"IE","irlandesa":"IE","holandes":"NL","holandesa":"NL",
    "belga":"BE","suizo":"CH","suiza":"CH","austriaco":"AT","austriaca":"AT","sueco":"SE","sueca":"SE","noruego":"NO","noruega":"NO",
    "finlandes":"FI","finlandesa":"FI","danes":"DK","danesa":"DK","estonio":"EE","leton":"LV","letona":"LV","lituano":"LT","lituana":"LT",
    "checo":"CZ","checa":"CZ","eslovaco":"SK","eslovaca":"SK","hungaro":"HU","hungara":"HU","griego":"GR","griega":"GR",
    "bulgaro":"BG","bulgara":"BG","rumano":"RO","rumana":"RO","croata":"HR","serbio":"RS","bosnio":"BA","esloveno":"SI",
    # LatAm
    "argentino":"AR","argentina":"AR","mexicano":"MX","mexicana":"MX","chileno":"CL","chilena":"CL","uruguayo":"UY","uruguaya":"UY",
    "paraguayo":"PY","paraguaya":"PY","peruano":"PE","peruana":"PE","boliviano":"BO","boliviana":"BO","colombiano":"CO","colombiana":"CO",
    "ecuatoriano":"EC","ecuatoriana":"EC","venezolano":"VE","venezolana":"VE","costarricense":"CR","panameno":"PA","panameña":"PA",
    "guatemalteco":"GT","guatemalteca":"GT","hondureno":"HN","hondureña":"HN","nicaraguense":"NI","salvadoreno":"SV","salvadoreña":"SV",
    "dominicano":"DO","dominicana":"DO","cubano":"CU","cubana":"CU","puertorriqueno":"PR","puertorriqueña":"PR",
}
EU    = {"ES","FR","DE","IT","PT","PL","GB","IE","NL","BE","CH","AT","SE","NO","FI","DK","EE","LV","LT","CZ","SK","HU","GR","BG","RO","HR","RS","BA","SI"}
LATAM = {"AR","MX","CL","UY","PY","PE","BO","CO","EC","VE","CR","PA","GT","HN","NI","SV","DO","CU","PR"}

def _region_of(code: str) -> str:
    if code in EU: return "Europe"
    if code in LATAM: return "LatAm"
    return np.nan

# --- flags 🇦🇷 → ISO2 ---
def _iso_from_flags(text: str):
    raw = str(text)
    vals = []
    for ch in raw:
        o = ord(ch)
        vals.append(o - 0x1F1E6 if 0x1F1E6 <= o <= 0x1F1FF else None)
    out, i = [], 0
    while i < len(vals)-1:
        if vals[i] is not None and vals[i+1] is not None:
            out.append(chr(vals[i]+65) + chr(vals[i+1]+65))
            i += 2
        else:
            i += 1
    return [c for c in out if c in (EU | LATAM)]

# --- patrones de auto-identificación ---
PAT_SELF = re.compile(r"\b(saludos(?:\s+desde)?|soy\s+de|somos\s+de|aqui\s+en|aquí\s+en|desde|reportando\s+desde)\b", re.I)
ALIAS_RX    = re.compile(r"\b(" + "|".join(re.escape(k) for k in sorted(ALIAS, key=len, reverse=True)) + r")\b", re.I)
DEMONYMS_RX = re.compile(r"\b(" + "|".join(re.escape(k) for k in sorted(DEMONYMS, key=len, reverse=True)) + r")\b", re.I)
HASHTAG_RX  = re.compile(r"#([a-zA-Záéíóúñü]{3,})")

# método → prioridad (más alto = mejor)
METHOD_PRI = {"flag":5,"self+alias":4,"self+demonym":3,"hashtag":2,"hashtag_demonym":1,"demonym":1,"none":0}
CONF_PRI   = {"high":3,"med":2,"low":1}

def detect_self_country(text: str):
    """
    Devuelve tupla (code, region, conf, method) o ('Unknown','Unknown','low','none')
    Prioridad: Flag > (self+alias) > (self+demonym) > hashtag > demonym
    """
    raw = str(text)
    norm = _norm(raw)

    flags = _iso_from_flags(raw)
    if flags:
        code = flags[0]
        return (code, _region_of(code), "high", "flag")

    if PAT_SELF.search(norm):
        m = ALIAS_RX.search(norm)
        if m:
            code = ALIAS[m.group(1)]
            return (code, _region_of(code), "high", "self+alias")
        m = DEMONYMS_RX.search(norm)
        if m:
            code = DEMONYMS[m.group(1)]
            return (code, _region_of(code), "med", "self+demonym")

    hashtags = [_norm(x) for x in HASHTAG_RX.findall(raw)]
    for h in hashtags:
        if h in ALIAS:
            code = ALIAS[h]
            return (code, _region_of(code), "med", "hashtag")
        if h in DEMONYMS:
            code = DEMONYMS[h]
            return (code, _region_of(code), "low", "hashtag_demonym")

    m2 = DEMONYMS_RX.search(norm)
    if m2:
        code = DEMONYMS[m2.group(1)]
        return (code, _region_of(code), "low", "demonym")

    return ("Unknown","Unknown","low","none")

# --- detección por comentario ---
tmp = df[["user_id","comment_id","comment_clean"]].copy()
vals = tmp["comment_clean"].fillna("").map(detect_self_country).tolist()
tmp[["code","region","conf","method"]] = pd.DataFrame(vals, index=tmp.index)

# conservamos solo países válidos (EU/LatAm)
valid = tmp[tmp["code"].isin(EU | LATAM)].copy()
if not valid.empty:
    # agregamos por usuario y país: conteo + mejores señales
    valid["conf_score"]   = valid["conf"].map(CONF_PRI).astype(int)
    valid["method_score"] = valid["method"].map(METHOD_PRI).astype(int)

    grp = (valid
           .groupby(["user_id","code","region"], as_index=False)
           .agg(
               n_mentions = ("comment_id","count"),
               max_conf   = ("conf_score","max"),
               max_method = ("method_score","max"),
           ))

    # para cada user elegimos el (code,region) con mayor (max_conf, n_mentions, max_method)
    grp.sort_values(["user_id","max_conf","n_mentions","max_method"],
                    ascending=[True, False, False, False], inplace=True)
    best = grp.groupby("user_id", as_index=False).first()[["user_id","code","region"]]
    best.rename(columns={"code":"user_country","region":"user_region"}, inplace=True)

    # merge al master
    df = df.merge(best, on="user_id", how="left")
else:
    # si no hay ninguna auto-identificación válida, asignamos NaN (no exportamos nada)
    df["user_country"] = np.nan
    df["user_region"]  = np.nan

print("Añadidas columnas en df: user_country (ISO-2), user_region (Europe/LatAm/None)")


Añadidas columnas en df: user_country (ISO-2), user_region (Europe/LatAm/None)


# Time Stats by User --> Bots & Core users

In [75]:
# 1 Métricas por usuario (tiempo + comportamiento)

# ===========================================
# Time Stats by User — métricas base por usuario
#  - Calcula: user_n_comments, user_days_active, user_freq_diaria,
#             mean_gap_h, burstiness, hour_var,
#             n_videos, n_channels, dup_ratio, url_rate, mention_rate,
#             emoji_frac, insulto_rate, n_insultos_mean, channel_entropy
#  - Crea 'user_feats' para la celda 2
# ===========================================


# 1) Parsing único de tiempo
df["comment_time"] = pd.to_datetime(df["comment_time"], errors="coerce", utc=True)
g = df.groupby("user_id", dropna=False)

# 2) Utilidades
URL_RX   = re.compile(r"http[s]?://|www\.", re.I)
MENT_RX  = re.compile(r"@[\w_]+")
EMOJI_RX = re.compile(r"[^\w\s,.\-¡!¿?\(\)\"'@#:/]")

def _frac_emojis(s: str) -> float:
    s = str(s)
    return len(EMOJI_RX.findall(s)) / max(1, len(s))

def _norm_text(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"\s+"," ", s).strip()
    return s

def _hash(s: str) -> str:
    return hashlib.md5(_norm_text(s).encode("utf-8")).hexdigest()

def _entropy(items) -> float:
    if not items:
        return 0.0
    c = Counter(items); tot = sum(c.values()); p = [v/tot for v in c.values()]
    return -sum(pi*log2(pi) for pi in p if pi > 0)

def _to_ns(series_dt: pd.Series) -> np.ndarray:
    arr = series_dt.dropna().sort_values().values
    return arr.astype("datetime64[ns]").astype("int64")

def _mean_gap_h(ns_arr: np.ndarray) -> float:
    n = ns_arr.size
    if n < 2:
        return np.nan
    gaps_h = np.diff(ns_arr) / (1e9 * 3600.0)
    return float(np.mean(gaps_h))

def _burst(ns_arr: np.ndarray) -> float:
    n = ns_arr.size
    if n < 3:
        return 0.0
    diffs = np.diff(ns_arr).astype("float64")
    mu = diffs.mean()
    if mu <= 0:
        return 0.0
    return float(diffs.std(ddof=0) / (mu + 1e-9))

def _hour_var(series_dt: pd.Series) -> float:
    hrs = series_dt.dropna().dt.hour.to_numpy()
    return float(np.var(hrs, ddof=0)) if hrs.size else 0.0

def user_days_active(series_dt: pd.Series) -> int:
    return int(series_dt.dropna().dt.floor("D").nunique())

# 3) Cálculos por usuario (vectorizados)
ns_arrays = g["comment_time"].apply(_to_ns)

user_feats = pd.DataFrame({
    "user_id": ns_arrays.index.astype(str),
    "mean_gap_h": ns_arrays.apply(_mean_gap_h).values,
    "burstiness": ns_arrays.apply(_burst).values,
    "hour_var": g["comment_time"].apply(_hour_var).values,
    "user_days_active": g["comment_time"].apply(user_days_active).values,
})

user_feats["user_n_comments"] = g.size().values
user_feats["user_freq_diaria"] = user_feats["user_n_comments"] / np.maximum(1, user_feats["user_days_active"])

# 4) Diversidad y señales de spam/estilo
hashes = g["comment_clean"].apply(lambda s: [_hash(x) for x in s])
user_feats["dup_ratio"] = hashes.apply(lambda h: 1.0 - (len(set(h)) / len(h) if len(h) else 1.0)).values

user_feats["n_videos"]   = g["video_title"].nunique().values
user_feats["n_channels"] = g["channel_title"].nunique().values

user_feats["url_rate"]     = g["comment"].apply(lambda s: float(np.mean([bool(URL_RX.search(str(x))) for x in s]))).values
user_feats["mention_rate"] = g["comment"].apply(lambda s: float(np.mean([bool(MENT_RX.search(str(x))) for x in s]))).values
user_feats["emoji_frac"]   = g["comment"].apply(lambda s: float(np.mean([_frac_emojis(x) for x in s]))).values

user_feats["insulto_rate"]    = g["insulto"].mean().values
user_feats["n_insultos_mean"] = g["n_insultos"].mean().values
user_feats["channel_entropy"] = g["channel_title"].apply(lambda s: _entropy(list(s))).values

# 5) Tipos numéricos consistentes
num_cols = user_feats.columns.difference(["user_id"])
user_feats[num_cols] = user_feats[num_cols].apply(pd.to_numeric, errors="coerce")



In [ ]:
# ===========================================
# Time Stats by User --> Bots & Segments (CELDA 2 actualizada)
#  - Usa 'user_feats' creado en Celda 1
#  - Crea: bot_flag, bot_score, user_segment, user_rank
#  - Merge al df y limpia intermedias si quedaron en df
# ===========================================


# ---------- 1) Cuantiles para reglas (sobre user_feats) ----------
Q = user_feats[["user_n_comments","n_channels","user_freq_diaria"]].quantile(
    [0.35, 0.50, 0.85, 0.90, 0.95, 0.97]
)
def q(p, col): return float(Q.loc[p, col])

# ---------- 2) Bot score (heurístico) + umbral dinámico (p98, conservador) ----------
acct_median = (
    df.groupby("user_id")["days_since_account_creation"]
      .median()
      .reindex(user_feats["user_id"])
      .fillna(365)
      .to_numpy()
)

spam_proxy = 0.5*user_feats["url_rate"] + 0.3*user_feats["dup_ratio"] + 0.2*user_feats["mention_rate"]

bot_score = (
    0.28*user_feats["dup_ratio"] +
    0.18*spam_proxy +
    0.12*(user_feats["user_n_comments"] >= 50).astype(float) +
    0.12*(user_feats["mean_gap_h"] <= 0.25).fillna(0).astype(float) +
    0.10*(user_feats["emoji_frac"] > 0.08).astype(float) +
    0.12*(user_feats["insulto_rate"] > 0.2).astype(float) +
    0.08*(acct_median < 30).astype(float)
).clip(0, 1)
user_feats["bot_score"] = bot_score

TH = max(0.55, float(user_feats["bot_score"].quantile(0.98)))
likely_bot_heur = (user_feats["bot_score"] >= TH)

# ---------- 3) IsolationForest (complementario; más estricto) ----------
iso_cols = [
    "dup_ratio","user_n_comments","n_videos","n_channels",
    "mean_gap_h","emoji_frac","insulto_rate","user_freq_diaria",
    "hour_var","channel_entropy"
]
X  = user_feats[iso_cols].replace([np.inf, -np.inf], np.nan).fillna(0).values
Xz = StandardScaler().fit_transform(X)

iso = IsolationForest(n_estimators=400, contamination=0.02, random_state=42)
likely_bot_iso = (iso.fit_predict(Xz) == -1)

user_feats["bot_flag"] = likely_bot_heur | likely_bot_iso

# ---------- 4) Segmentación (ACTIVO; ESPORÁDICO = 1 comentario) ----------
seg = np.full(len(user_feats), "activo", dtype=object)  # base más amplia

# 4.1 sospecha_bot (prioridad máxima)
seg = np.where(user_feats["bot_flag"], "sospecha_bot", seg)

# 4.2 núcleo_duro
mask_core = (
    (user_feats["user_n_comments"] >= q(0.97, "user_n_comments")) &
    ((user_feats["n_channels"] >= q(0.85, "n_channels")) | (user_feats["user_freq_diaria"] >= q(0.95, "user_freq_diaria")))
)
seg = np.where((seg != "sospecha_bot") & mask_core, "nucleo_duro", seg)

# 4.3 fiel
mask_fiel = (
    (user_feats["user_n_comments"] >= q(0.85, "user_n_comments")) &
    (user_feats["n_channels"] <= q(0.35, "n_channels"))
)
seg = np.where((seg == "activo") & mask_fiel, "fiel", seg)

# 4.4 esporádico = 1 comentario (y que no sea bot/core/fiel)
mask_espo = (user_feats["user_n_comments"] == 1)
seg = np.where((seg == "activo") & mask_espo, "esporadico", seg)

user_feats["user_segment"] = seg

# ---------- 5) Ranking por actividad ----------
user_feats["user_rank"] = user_feats["user_n_comments"].rank(method="dense", ascending=False).astype(int)

# ---------- 6) Merge mínimo al df (sin sufijos) ----------
final_cols = ["user_id","bot_flag","bot_score","user_segment","user_rank",
              "user_n_comments","user_days_active","user_freq_diaria"]

df = df.drop(columns=[c for c in final_cols if c in df.columns and c != "user_id"], errors="ignore")
df = df.merge(user_feats[final_cols], on="user_id", how="left", validate="m:1")

# ---------- 7) Limpieza de posibles intermedias en df ----------
drop_if_present = [
    "mean_gap_h","burstiness","hour_var","n_videos","n_channels",
    "dup_ratio","url_rate","mention_rate","emoji_frac","insulto_rate",
    "n_insultos_mean","channel_entropy"
]
df.drop(columns=[c for c in drop_if_present if c in df.columns], inplace=True, errors="ignore")


In [77]:
col = "bot_flag"

conteos = df[col].value_counts()
total = conteos.sum()

print(f"Conteo de valores para '{col}':\n")
for valor, count in conteos.items():
    porcentaje = count / total * 100
    print(f"{valor:<15} {count:>5} ({porcentaje:5.2f}%)")

Conteo de valores para 'bot_flag':

0               97203 (85.58%)
1               16380 (14.42%)


# Classificación -1 a 1 de sentimientos

In [4]:
# 3. Función que mapea a -1, 0, 1 según el 'compound'
def sentiment_label_vader(text, low= -0.05, high=0.05):
    s = sia.polarity_scores(text)['compound']
    if s >= high: return  1
    if s <= low:  return -1
    return 0

In [5]:
df['sentiment'] = df['comment'].astype(str).apply(sentiment_label_vader)

In [6]:
# Conteos absolutos y relativos
counts = df['sentiment'].value_counts().sort_index()
props  = df['sentiment'].value_counts(normalize=True).sort_index() * 100

print("Counts:\n", counts)
print("\nPercentages:\n", props.round(2))


Counts:
 sentiment
-1    34969
 0    72563
 1     6244
Name: count, dtype: int64

Percentages:
 sentiment
-1    30.73
 0    63.78
 1     5.49
Name: proportion, dtype: float64


In [7]:
for s in [-1, 0, 1]:
    print(f"\n=== Clase {s} ===")
    ejemplos = df[df['sentiment']==s]['comment'].sample(5, random_state=42).tolist()
    for c in ejemplos:
        print("-", c)



=== Clase -1 ===
- Miguel 👏👏 estoy impactada 😢😢😢 😞😔 no puedo ni hablar.
- No entiendo nada por el canal prensa alternativa de Juan José del castillo publican que Rusia 🇷🇺 está acabando con los rebeldes los mig 31 y sucoy 35 están bombardeado sin piedad a los extremistas kurdos y por aquí dicen lo contrario así que no entiendo nada
- TU ERES PROCOMUNISMO JAJAJAJAJA YA VI VARIOS DE TUS VIDEOS Y TODOS USTEDES HACEN POSTURAS DE CONDOLENCIAS A GOBERNANTES COMUNISTAS, POBRE Y LAMENTABLE, PERO ESTA ES LA GRAN PRUEBA PARA LA HUMANIDAD EL COMUNISMO!!!!!, AL FINAL ESTO ES UNA BATALLA ESPIRITUAL Y DE UNA VES DE AVISO EL COMUNISMO YA PERDIO Y TU PARECE SER QUE TE INCLINAS POR UN BANDO PERDEDOR!
- No todo van a ser malas noticias para Rusia dice...y si no fuera por la OTAN Ucrania se mea encima y aun así solo esta consiguiendo una mierda bien gorda.
- Estos cretinos no son concientes que están abriendo las puertas del INFIERNO para la humanidad, los ciudadanos europeos deben rechazar este tipo de 

In [8]:
tabla = (
    df
    .groupby(['condiciones_cuenta', 'sentiment'])
    .size()
    .unstack(fill_value=0)
)
print(tabla)

sentiment              -1      0     1
condiciones_cuenta                    
noticiero            7862  17333  1214
pro-ruso            16776  35988  2869
pro-ucraniano       10331  19242  2161


In [9]:
tabla_pct = tabla.div(tabla.sum(axis=1), axis=0).round(3)*100
print(tabla_pct)

sentiment             -1     0    1
condiciones_cuenta                 
noticiero           29.8  65.6  4.6
pro-ruso            30.2  64.7  5.2
pro-ucraniano       32.6  60.6  6.8


-----------------------

# Arguments classification

In [105]:

TXT = "comment_clean" if "comment_clean" in df.columns else "comment"

def _norm(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"\brussia\b", "rusia", s)
    s = re.sub(r"\botar\b", "otan", s)
    s = re.sub(r"\bee\s*\.?\s*uu\s*\.?\b", "eeuu", s)
    s = re.sub(r"\bu\.?\s*e\.?\b", "ue", s)
    s = re.sub(r"\bnato\b", "otan", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

text = df[TXT].astype(str).map(_norm)
lbl  = df["label_final"].astype(str).str.lower().fillna("")
chan = df["condiciones_cuenta"].astype(str).str.lower().fillna("")

def channel_side(x: str) -> str:
    if "pro-ruso" in x: return "ruso"
    if "pro-ucraniano" in x: return "ucraniano"
    return "neutro"
chan_side = chan.map(channel_side)

# -------- utilidades de proximidad --------
TOKEN_SPLIT_RX = re.compile(r"[^\wáéíóúüñ]+", re.I)
def split_sentences(s: str): return re.split(r"[.!?¡¿\n\r]+", s)

def has_proximity(sent: str, terms_a, terms_b, win=6) -> bool:
    toks = [t for t in TOKEN_SPLIT_RX.split(sent) if t]
    if not toks: return False
    pos_a = [i for i, t in enumerate(toks) if t in terms_a]
    pos_b = [i for i, t in enumerate(toks) if t in terms_b]
    if not pos_a or not pos_b: return False
    for i in pos_a:
        for j in pos_b:
            if abs(i - j) <= win:
                return True
    return False

def any_sentence(sentences, predicate):
    for s in sentences:
        if predicate(s): return True
    return False

# -------- léxicos --------
CREATOR_TERMS = {
    "miguel","borja","memorias","pez","memoriasdepez","canal","video","vídeo","creador","youtuber",
    "tu","tus","te","usted","ustedes","vos","vosotros"  # para proximidad con 2ª persona
}
CREATOR_STRONG = {"miguel","borja","memorias","memoriasdepez","canal"}

THANK_TERMS = {"gracias","agradezco","agradecemos","agradecida","agradecido","milgracias","muchasgracias"}
VIDEO_TERMS  = {"video","vídeo","analisis","análisis","directo","contenido","trabajo"}

# insulto explícito (como antes)
INSULT_TERMS = {
    "basura","vendido","vendida","vendidos","vendidas","payaso","payasa","payasos","payasas",
    "mentiroso","mentirosa","mentirosos","mentirosas","propagandista","charlatan","charlatán",
    "estafa","manipulador","manipuladora","manipuladores","desinformador","desinformadora",
    "relatero","relatera","relateros","relato"
}

# NUEVO: léxico negativo/crítico no necesariamente insultante
NEG_CREATOR_TERMS = {
    "relato","relatos","sesgado","sesgada","sesgados","sesgadas","tendencioso","tendenciosa",
    "clickbait","click","bait","falso","falsa","falsedad","fake","engañoso","engañosa","engaño",
    "mientes","miente","mentis","mentís","mentir","no sabes","no sabés","no tienes idea",
    "pésimo","pesimo","malo","malisimo","malísimo","pobre análisis","mal análisis",
    "desinformar","desinformas","desinformás","desinformacion","desinformación","manipulas","manipulás",
    "panfleto","propaganda","trola","verso"
}
NEG_CREATOR_RX = [
    re.compile(r"\b(p[eé]simo|mal[ií]simo|malo)\s+(video|an[aá]lisis)\b"),
    re.compile(r"\b(gracias por el relato)\b"),
    re.compile(r"\b(ment[ií]s|mientes|mentiroso)\b"),
    re.compile(r"\b(desinform(a|ás|as|ar))\b"),
    re.compile(r"\b(sesgad[oa]s?|tendencios[oa]s?)\b"),
    re.compile(r"\b(click\s?bait|clickbait)\b"),
]

# sarcasmos para filtrar "agradecimientos"
THANK_SARCASM_RX = [
    re.compile(r"\bgracias a (usa|eeuu|otan|occidente|ue)\b"),
    re.compile(r"\bgracias por el relato\b"),
    re.compile(r"\b(me )?hacen.*risa.*gracias\b"),
]

# --- otros ejes (igual que versión previa estricta) ---
def any_rx(s: str, rxs) -> bool:
    return any(rx.search(s) for rx in rxs)

RX_HIST = [
    re.compile(r"\bhistori[ac]a?\b", re.I),
    re.compile(r"\bcontexto hist[oó]rico\b", re.I),
    re.compile(r"\bdesde (el|la) (año|siglo)\b", re.I),
    re.compile(r"\bprecedente\b|\bcomparaci[oó]n\b", re.I),
    re.compile(r"\b(2014|maidan|donb[aá]s)\b", re.I),
]
RX_MEMES = [
    re.compile(r"[😂😅🤣]+"),
    re.compile(r"\bjaja(ja)+\b|\bjeje(je)+\b", re.I),
    re.compile(r"\bmem(e|es)\b|\bchiste(s)?\b|\bc[oó]mico\b", re.I),
    re.compile(r"\bsevero loot\b|\bcope\b|\bcringe\b", re.I),
]
RX_PODER_RU = [
    re.compile(r"\bpotencia (militar|industrial)\b", re.I),
    re.compile(r"\bsuperioridad rusa\b|\barmas rusas\b|\bindustria rusa\b", re.I),
    re.compile(r"\bcapacidad (bélica|rusa)\b", re.I),
    re.compile(r"\bavanza (rusia|ej[eé]rcito ruso)\b|\brusia (es|est[aá]) (fuerte|superior)\b", re.I),
]
RX_CULPA_OTAN = [
    re.compile(r"\bculpa (de )?(la )?(otan|occidente|ue|eeuu|usa)\b", re.I),
    re.compile(r"\boccidente (provoc[oó]|provoca|empuj[oó])\b", re.I),
    re.compile(r"\bexpansi[oó]n de la otan\b|\bsanci[oó]n(es)?\b", re.I),
]
RX_CULPA_RU = [
    re.compile(r"\bculpa (de )?rusia\b|\brusia es (el|la) (agresor|culpable)\b", re.I),
    re.compile(r"\binvasi[oó]n rusa\b|\bcr[ií]menes? de guerra (ruso|rusos)\b", re.I),
]
RX_DEF_RU = [
    re.compile(r"\b(rusia|putin) (tiene|ten[ií]a) raz[oó]n\b", re.I),
    re.compile(r"\bdefend(er|iendo) a rusia\b", re.I),
    re.compile(r"\boperaci[oó]n especial (justa|leg[ií]tima)\b|\bdesnazificaci[oó]n\b|\bdesnazificar\b", re.I),
]
RX_DEF_UA = [
    re.compile(r"\bucrania (se defiende|resiste|tiene raz[oó]n)\b", re.I),
    re.compile(r"\bdefender a ucrania\b|\bresistencia ucraniana\b|\bderecho a defenderse\b", re.I),
]
RX_NEUTRAL = [
    re.compile(r"\bneutral(es)?\b|\bimparcial(es)?\b|\bobjetiv[oa]s?\b", re.I),
    re.compile(r"\b(no se sabe|no (se )?puede saber|ambos lados|ning[uú]n lado|ni rusia ni ucrania)\b", re.I),
]

def insults_at_creator_extended(s: str, insult_flag: bool, lab: str, chs: str) -> bool:
    """Marca insultos_al_creador si:
       (A) insulto explícito + proximidad a creador/video (ventana corta), o
       (B) bando comentario ≠ bando canal y hay crítica/agresión cerca de creador/video
           (léxico negativo o patrones de crítica), incluso sin insulto.
    """
    sents = split_sentences(s)

    # A) insulto explícito cerca del creador/video
    def _direct_insult(sent):
        return has_proximity(sent, CREATOR_STRONG, INSULT_TERMS, win=3) or \
               has_proximity(sent, CREATOR_TERMS, INSULT_TERMS, win=3)
    if insult_flag and any_sentence(sents, _direct_insult):
        # si además hay "gracias" + creador en la MISMA oración, lo descartamos (posible ironía ambigua)
        def _grat_near_creator(sent):
            return ("gracias" in sent) and has_proximity(sent, CREATOR_TERMS, THANK_TERMS | VIDEO_TERMS, win=4)
        if not any_sentence(sents, _grat_near_creator):
            return True

    # B) crítica/ataque no insultante hacia creador/video, pero con desacople de bando
    if lab != chs:
        def _negative_creator(sent):
            near_neg_lex = has_proximity(sent, CREATOR_TERMS | VIDEO_TERMS, NEG_CREATOR_TERMS, win=5)
            neg_rx_hit   = any(rx.search(sent) for rx in NEG_CREATOR_RX)
            return near_neg_lex or neg_rx_hit
        if any_sentence(sents, _negative_creator):
            return True

    return False

def thanks_to_creator(s: str) -> bool:
    if "gracias" not in s: return False
    if any(rx.search(s) for rx in THANK_SARCASM_RX): return False
    sents = split_sentences(s)
    def _grat(sent):
        has_thanks = has_proximity(sent, THANK_TERMS, CREATOR_TERMS | VIDEO_TERMS, win=5)
        return has_thanks or re.search(r"\bgracias (miguel|borja|por (el )?(video|an[aá]lisis))\b", sent)
    return any_sentence(sents, _grat)

def any_rx(s: str, rxs) -> bool: return any(rx.search(s) for rx in rxs)

def pick_axis_row(s: str, label: str, chs: str, insult_flag: bool) -> str|None:
    # 1) insulto/ataque al creador (ampliado)
    if insults_at_creator_extended(s, insult_flag, label, chs):
        return "insultos_al_creador"
    # 2) agradecimiento real al creador
    if thanks_to_creator(s):
        return "agradecimientos_al_creador"
    # 3) neutralidad condicionada
    if label == "neutro" and any_rx(s, RX_NEUTRAL):
        return "neutralidad"
    # 4) temáticos
    if any_rx(s, RX_HIST):       return "justificacion_historica"
    if any_rx(s, RX_MEMES):      return "festejos_y_memes"
    if any_rx(s, RX_PODER_RU):   return "poderio_ruso"
    if any_rx(s, RX_CULPA_OTAN): return "culpa_de_la_otan_occidente"
    if any_rx(s, RX_CULPA_RU):   return "culpa_de_rusia"
    if any_rx(s, RX_DEF_RU):     return "defensa_acciones_rusas"
    if any_rx(s, RX_DEF_UA):     return "defensa_acciones_ucranianas"
    return None

df["eje_argumentativo"] = [
    pick_axis_row(s, l, c, bool(ins))
    for s, l, c, ins in zip(text.values, lbl.values, chan_side.values, df["insulto"].values)
]

print("df['eje_argumentativo'] recalculado (insultos_al_creador ampliado con crítica/sarcasmo cerca del creador/video y desacople de bando).")

df['eje_argumentativo'] recalculado (insultos_al_creador ampliado con crítica/sarcasmo cerca del creador/video y desacople de bando).


# Unigrams & Bigrams

In [ ]:
# ===========================================
# Unigramas / Bigrams para Power BI (rápido + filtro de ruido)
#  - Scopes: label_final y channel_title
#  - Quita risas/muletillas ("jajaja", "bla bla", "xd", etc.)
#  - Mantiene dominio (rusia, ucrania, otan, eeuu) y normaliza alias
#  - Bigramas exportados con ESPACIO (no guion bajo)
#  - Salida: ../data/processed/pbi_unigrams.csv / pbi_bigrams.csv
# ===========================================
import pandas as pd, numpy as np, re
from pathlib import Path
from collections import Counter, defaultdict

OUT_DIR = Path("../data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)
UNI_PATH = OUT_DIR / "pbi_unigrams.csv"
BI_PATH  = OUT_DIR / "pbi_bigrams.csv"

TXT_COL = "comment_clean" if "comment_clean" in df.columns else "comment"
TXT     = df[TXT_COL].astype(str).values
LANGS   = (df["lang"].astype(str).values if "lang" in df.columns else np.array(["es"]*len(df)))

LABELS  = df["label_final"].fillna("desconocido").astype(str).values
CHANNEL = df["channel_title"].astype(str).values

# --- Stopwords funcionales (sin dominio)
stop_es = {
 "a","al","algo","algunas","algunos","ante","antes","como","con","contra","cual","cuando","de","del","desde","donde",
 "durante","e","el","ella","ellas","ellos","en","entre","era","erais","eran","eras","eres","es","esa","esas","ese","eso",
 "esos","esta","estaba","estaban","estado","estais","estamos","estan","estar","este","esto","estos","estoy","fin","fue",
 "fueron","fui","fuimos","ha","habeis","haber","habia","habla","hablan","hace","hacia","han","hasta","hay","la","las",
 "le","les","lo","los","mas","me","mi","mis","mucho","muy","nada","ni","no","nos","nosotros","o","os","otra","otras",
 "otro","otros","para","pero","poco","por","porque","que","quien","quienes","se","sea","segun","ser","si","sin","sobre",
 "sois","solamente","solo","somos","son","soy","su","sus","tal","tambien","tanto","te","tenia","tendra","teneis","tenemos",
 "tienen","toda","todas","todo","todos","tu","tus","un","una","uno","unos","vosotros","y","ya","q","xq","pq"
}
stop_en = {
 "a","an","the","and","or","but","to","of","in","on","for","with","as","by","is","are","was","were","be","been","being",
 "at","from","that","this","these","those","it","its","i","you","he","she","they","we","me","him","her","them","us",
 "my","your","his","her","their","our","not","no","do","does","did","have","has","had","so","if","just","about","into",
 "over","than","then","there","here","out","up","down","yes","very","also"
}
stop_pt = {"de","da","do","das","dos","em","no","na","nos","nas","e","ou","mas","para","por","com","sem",
           "um","uma","uns","umas","se","que","como","muito","muita","muitos","muitas","já","também"}
STOP_ADD = {
 "más","esta","está","están","estar","estoy","estaba","estaban",
 "va","van","ver","vez","así","tan","cada","mejor","dice","decir",
 "hacer","puede","tiene","siempre","ahora","bien","yo","tu","usted",
 "años","gente","si","qué"
}
def _pick_stop(lang: str):
    lang = (lang or "es").lower()
    if   lang.startswith("en"): base = stop_en
    elif lang.startswith("pt"): base = stop_pt
    else:                       base = stop_es
    return base | STOP_ADD

# --- Normalización de alias
ALIAS = {
    "usa":"eeuu","estados_unidos":"eeuu",
    "nato":"otan",
    "union_europea":"ue",
    "zelenski":"zelensky","zelenskyy":"zelensky","zelenskyi":"zelensky",
    "ucranianos":"ucraniano","ucranianas":"ucraniano",
    "rusos":"ruso","rusa":"ruso","rusas":"ruso",
    "israelíes":"israel","estadounidenses":"eeuu"
}
def normalize_term(t: str) -> str:
    t = str(t).strip().lower().replace(" ", "_")
    return ALIAS.get(t, t)

# --- Filtros de ruido (risas/muletillas)
LAUGH_RX   = re.compile(r'^(?:j[aeiou]|aj|ha|he|hi|ho){3,}$', re.I)   # ej: jajaja, jejeje, ajajaja, hahaha
XD_RX      = re.compile(r'^(?:x+d+|d+x+)$', re.I)                     # xd, xdd, dxx
NOISE_BASE = {"jaja","jajaja","jajajaja","jeje","jejeje","ajaj","ajajaja","xd","lol","lmao","bla","blabla","bla_bla"}

def is_noise_token(t: str) -> bool:
    t = t.lower()
    return (t in NOISE_BASE) or bool(LAUGH_RX.match(t)) or bool(XD_RX.match(t))

# --- Tokenización
tok_rx = re.compile(r"[^\wáéíóúüñ]+", re.I)

# cache stopwords
_stop_cache = {}
def get_stop(lang):
    if lang not in _stop_cache:
        _stop_cache[lang] = _pick_stop(lang)
    return _stop_cache[lang]

def tokenize(text: str, lang: str):
    st = get_stop(lang)
    toks = []
    for t in tok_rx.split(text.lower()):
        if not t or len(t) < 3 or t.isdigit(): 
            continue
        if t in st: 
            continue
        if is_noise_token(t):
            continue
        toks.append(normalize_term(t))
    return toks

def to_bigrams(tokens):
    # bigrama con ESPACIO, y filtramos bigrams ruidosos (ambos lados ruido o frase tipo "bla bla")
    bigs = []
    for a, b in zip(tokens, tokens[1:]):
        if is_noise_token(a) and is_noise_token(b):
            continue
        term = f"{a.replace('_',' ')} {b.replace('_',' ')}"
        if term.strip() in {"bla bla"}:
            continue
        bigs.append(term)
    return bigs

# ===========================
# STREAMING COUNT (una pasada)
# ===========================
uni_label_cnt = defaultdict(Counter)
bi_label_cnt  = defaultdict(Counter)
uni_chan_cnt  = defaultdict(Counter)
bi_chan_cnt   = defaultdict(Counter)
docs_label = Counter()
docs_chan  = Counter()

for text, lang, lbl, ch in zip(TXT, LANGS, LABELS, CHANNEL):
    toks = tokenize(text, lang)
    bigs = to_bigrams(toks) if toks else []
    uni_label_cnt[lbl].update(toks)
    bi_label_cnt[lbl].update(bigs)
    docs_label[lbl] += 1
    uni_chan_cnt[ch].update(toks)
    bi_chan_cnt[ch].update(bigs)
    docs_chan[ch]  += 1

def counters_to_df(cmap, scope_type: str, docs_counter: Counter):
    rows = []
    for scope_val, cnt in cmap.items():
        total = sum(cnt.values())
        if total == 0:
            continue
        k = 0
        n_docs = int(docs_counter[scope_val])
        for term, n in cnt.most_common():
            k += 1
            rows.append((scope_type, scope_val, term, n,
                         round(100.0*n/total, 4), k, n_docs))
    return pd.DataFrame(rows, columns=["scope_type","scope_value","term","n","pct","rank","n_docs"])

uni_label_df = counters_to_df(uni_label_cnt, "label",   docs_label)
bi_label_df  = counters_to_df(bi_label_cnt,  "label",   docs_label)
uni_chan_df  = counters_to_df(uni_chan_cnt,  "channel", docs_chan)
bi_chan_df   = counters_to_df(bi_chan_cnt,   "channel", docs_chan)

pbi_unigrams = pd.concat([uni_label_df, uni_chan_df], ignore_index=True).sort_values(["scope_type","scope_value","rank"])
pbi_bigrams  = pd.concat([bi_label_df,  bi_chan_df],  ignore_index=True).sort_values(["scope_type","scope_value","rank"])

pbi_unigrams.to_csv(UNI_PATH, index=False, encoding="utf-8-sig")
pbi_bigrams.to_csv(BI_PATH,  index=False, encoding="utf-8-sig")

print(f"Guardado:\n - {UNI_PATH}\n - {BI_PATH}")
print("Ejemplo UNIGRAMS:", pbi_unigrams.head(10).to_string(index=False))
print("Ejemplo BIGRAMS :", pbi_bigrams.head(10).to_string(index=False))


Guardado:
 - ..\data\processed\pbi_unigrams.csv
 - ..\data\processed\pbi_bigrams.csv
Ejemplo UNIGRAMS: scope_type scope_value    term   n    pct  rank  n_docs
   channel     EL PAÍS   rusia 773 2.5083     1    3493
   channel     EL PAÍS ucrania 597 1.9372     2    3493
   channel     EL PAÍS  guerra 466 1.5121     3    3493
   channel     EL PAÍS    ruso 417 1.3531     4    3493
   channel     EL PAÍS    otan 350 1.1357     5    3493
Ejemplo BIGRAMS : scope_type scope_value           term  n    pct  rank  n_docs
   channel     EL PAÍS estados unidos 59 0.2156     1    3493
   channel     EL PAÍS     viva rusia 37 0.1352     2    3493
   channel     EL PAÍS guerra mundial 25 0.0913     3    3493
   channel     EL PAÍS  rusia ucrania 23 0.0840     4    3493
   channel     EL PAÍS  propio pueblo 23 0.0840     5    3493


# Terminos específicos

In [ ]:

TXT = "comment_clean" if "comment_clean" in df.columns else "comment"

# Lista ordenada por prioridad (Wagner, Kursk, Prigozhin arriba)
TERMS_ORDERED = [
    ("Wagner",         [r"\bwagner\b"]),
    ("Kursk",          [r"\bkursk\b"]),
    ("Prigozhin",      [r"\bprigozhin\b|prigogin|prigojin|prigogine"]),
    ("Bucha",          [r"\bbucha\b"]),
    ("Azovstal",       [r"\bazovstal\b"]),
    ("Bakhmut",        [r"\bbakhmut\b|bah?mut"]),
    ("Avdiivka",       [r"\bavdi(iv)?ka\b|avdi(iv)?ka"]),
    ("Kakhovka",       [r"\bkakhovka\b|cajovka|kajovka|kak?hovka"]),
    ("Kerch_Bridge",   [r"\b(kerch|crime[ao]\s*bridge|puente\s+de\s+crimea|puente\s+de\s+kerch)\b", r"\bkerc[hx]\b"]),
    ("HIMARS",         [r"\bhimars?\b"]),
    ("ATACMS",         [r"\batacms?\b"]),
    ("Leopard",        [r"\bleopard\b"]),
    ("Abrams",         [r"\babrams\b"]),
    ("Patriot",        [r"\bpatriot\b"]),
    ("StormShadow",    [r"\bstorm(?:\s|-)?shadow\b"]),
    ("Taurus",         [r"\btaurus\b"]),
    ("Shahed_Geran",   [r"\bshaheds?\b|\bgeran-?\d*\b"]),
    ("Kinzhal",        [r"\bkinzhal\b"]),
    ("Kalibr",         [r"\bkalibr\b|\bcalibre\b"]),
    ("Iskander",       [r"\biskander\b"]),
    ("Zaporizhzhia",   [r"\bzapo?riz(h|j)(zh|z)ia\b|\bzapo?ro?ri?a\b"]),
    ("Kherson",        [r"\bkherson\b|jerson|gerson"]),
    ("Kharkiv",        [r"\bkharkiv\b|jarkov|kharkov|jarkiv"]),
    ("Donbas",         [r"\bdonb(a|á)s\b|donbass"]),
    ("Lyman",          [r"\blyman\b"]),
    ("Severodonetsk",  [r"\bseverodonets?k\b"]),
    ("Lysychansk",     [r"\blysychansk\b|lisichansk"]),
    ("Kupiansk",       [r"\bkupians?k\b"]),
    ("Pokrovsk",       [r"\bpokrovsk\b"]),
    ("Mariupol",       [r"\bmariupol\b|mari[uú]pol"]),
    ("Crimea",         [r"\bcrimea\b|crim[eé]a|krimea"]),
    ("Belgorod",       [r"\bbelgorod\b|b[eé]lgorod"]),
    ("AZOV",           [r"\bazov\b"]),
    ("NAFO",           [r"\bnafo\b"]),
    ("ucranazi",       [r"\bucranazis?\b"]),
]

# Compilación de patrones
TERMS_COMPILED = [(label, re.compile("|".join(pats), re.I)) for label, pats in TERMS_ORDERED]

def _first_special_term(text: str):
    s = str(text)
    for label, rx in TERMS_COMPILED:
        if rx.search(s):
            return label
    return np.nan

df["term_especial"] = df[TXT].apply(_first_special_term)

print("Columna df['term_especial'] creada/actualizada (un único término por prioridad).")

Columna df['term_especial'] creada/actualizada (un único término por prioridad).


# Relevant events 2024 assignation

In [ ]:
# ==========================================================
# FIX: merge_asof con ambas claves tz-aware (datetime64[ns, UTC])
# Asigna último evento 2024 <= fecha del video (mismo día permitido)
# Crea/actualiza: evento, evento_fecha, relacion_evento, sub_tipo_evento, tipo_evento
# ==========================================================


# 1) Catálogo 2024 en el orden provisto (SIN .values para no perder tz)
events_2024_ordered = [
    ("Bombardeo en mercado de Donetsk (civiles muertos)",   "2024-01-21", "militar",     "simbolico",   "pro-ruso"),
    ("Muerte de Alexei Navalny en prisión",                 "2024-02-16", "politico",    "simbolico",   "pro-ucraniano"),
    ("Caída de Avdiivka",                                   "2024-02-18", "militar",     "simbolico",   "pro-ruso"),
    ("Paquete de ayuda de EE. UU. (61 mil millones USD)",   "2024-04-20", "geopolitico", "estrategico", "pro-ucraniano"),
    ("Ofensiva rusa en el óblast de Járkov",                "2024-05-10", "militar",     "estrategico", "pro-ruso"),
    ("Ataque ruso a hospital infantil en Kiev (Okhmatdyt)", "2024-07-08", "militar",     "simbolico",   "pro-ucraniano"),
    ("Incursión ucraniana en región rusa de Kursk",         "2024-08-06", "militar",     "estrategico", "pro-ucraniano"),
    ("Toma de Vuhledar (Donetsk)",                          "2024-10-02", "militar",     "estrategico", "pro-ruso"),
    ("Mes de mayores avances rusos (~200 km²)",             "2024-10-25", "militar",     "estrategico", "pro-ruso"),
    ("Ataque masivo con drones ucranianos sobre Moscú",     "2024-11-10", "militar",     "simbolico",   "pro-ucraniano"),
    ("Caída del régimen sirio (Assad huye a Moscú)",        "2024-12-08", "geopolitico", "simbolico",   "pro-ucraniano"),
]
ev = pd.DataFrame(events_2024_ordered,
                  columns=["evento","evento_fecha","relacion_evento","sub_tipo_evento","tipo_evento"])
ev["evento_fecha"] = pd.to_datetime(ev["evento_fecha"], utc=True)
ev = ev.sort_values("evento_fecha").reset_index(drop=True)  # conserva tz

# 2) Fecha de video (dataset ya filtrado a 2024 y no nulo)
df["_video_dt"] = pd.to_datetime(df["video_published_at"], errors="raise", utc=True)

# 3) Asignación vectorizada: último evento <= fecha del video
df_sorted = df.sort_values("_video_dt")
asof = pd.merge_asof(
    df_sorted[["_video_dt"]],
    ev, left_on="_video_dt", right_on="evento_fecha",
    direction="backward", allow_exact_matches=True
)

# 4) Volcar columnas al master (alineando por índice tras ordenar)
df.loc[df_sorted.index, "evento"]           = asof["evento"].values
df.loc[df_sorted.index, "evento_fecha"]     = asof["evento_fecha"].values
df.loc[df_sorted.index, "relacion_evento"]  = asof["relacion_evento"].values
df.loc[df_sorted.index, "sub_tipo_evento"]  = asof["sub_tipo_evento"].values
df.loc[df_sorted.index, "tipo_evento"]      = asof["tipo_evento"].values

# 5) Limpieza auxiliar
df.drop(columns=["_video_dt"], inplace=True)

print("Eventos 2024 asignados por 'merge_asof' (último evento ≤ fecha del video) con claves tz-aware.")


Eventos 2024 asignados por 'merge_asof' (último evento ≤ fecha del video) con claves tz-aware.


In [121]:
# Guardar dataset limpio en la carpeta del proyecto
output_path = "../data/processed/8_final_master_enriched.csv"
df.to_csv(output_path, index=False)

# BI Tool Star Tables Export

In [ ]:
# ============================================================
# GOLD / BI LAYER para Power BI — Star Schema desde df actual (fix video_key)
#   Salida: ../data/bi_layer/*.csv (UTF-8 BOM)
#   Grano del fact: comment_id
#   FKs: user_id, channel_id, video_key, evento
#   Mejora: dim_date continua (TZ local) + comment_date para relación
# ============================================================

# Config mínima
LOCAL_TZ    = "Europe/Madrid"  # eje temporal del reporte
BUFFER_DAYS = 7                # margen para rolling/edges

OUT_DIR = Path("../data/bi_layer")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def _hash_series(parts_df: pd.DataFrame) -> pd.Series:
    # Vectorizado: concatena como string estable y aplica md5
    s = parts_df.astype(str).agg("|".join, axis=1)
    return s.map(lambda x: hashlib.md5(x.encode("utf-8")).hexdigest())

def _to_local_floor_day(utc_ts: pd.Series, tz: str) -> pd.Series:
    # Convierte UTC -> tz local, baja a día y quita tz (naive date)
    return (
        pd.to_datetime(utc_ts, utc=True, errors="coerce")
          .dt.tz_convert(tz)
          .dt.floor("D")
          .dt.tz_localize(None)
    )

# --- Normalizaciones mínimas de tiempo/llaves (sin cambiar lógicas previas)
df["comment_time"]       = pd.to_datetime(df["comment_time"], errors="coerce", utc=True)
df["video_published_at"] = pd.to_datetime(df["video_published_at"], errors="coerce", utc=True)
if pd.api.types.is_datetime64_any_dtype(df["evento_fecha"]) and getattr(df["evento_fecha"].dt, "tz", None) is None:
    df["evento_fecha"] = pd.to_datetime(df["evento_fecha"], errors="coerce").dt.tz_localize("UTC")

# --- video_key auxiliar (vectorizado y estable)
df["_video_key"] = _hash_series(df[["channel_id","video_title","video_published_at"]])

# --- Agregados por usuario que faltaban en el master
user_agg = (
    df.groupby("user_id", as_index=False)
      .agg(n_channels_user=("channel_id","nunique"),
           n_videos_user=("_video_key","nunique"))
)

# ======================
# DIM USER
# ======================
base_user_cols = [
    "user_id","user_name","user_region","user_country",
    "user_segment","user_rank","bot_flag","bot_score",
    "user_n_comments","user_days_active","user_freq_diaria"
]
dim_user = (
    df.sort_values(["user_id","comment_time"])
      .drop_duplicates("user_id")[base_user_cols]
      .merge(user_agg, on="user_id", how="left", validate="1:1")
      .rename(columns={"n_channels_user":"n_channels",
                       "n_videos_user":"n_videos"})
      .reset_index(drop=True)
)

# ======================
# DIM CHANNEL
# ======================
dim_channel = (
    df.sort_values(["channel_id","comment_time"])
      .drop_duplicates("channel_id")[["channel_id","channel_title","subscriber_count","condiciones_cuenta"]]
      .reset_index(drop=True)
)

# ======================
# DIM VIDEO
# ======================
video_base = (
    df[["channel_id","video_title","video_published_at","video_views","video_likes","video_duration","video_category_id","video_tags","_video_key"]]
      .drop_duplicates("_video_key")
      .copy()
      .rename(columns={"_video_key":"video_key"})
)
dim_video = video_base[
    ["video_key","channel_id","video_title","video_published_at","video_views","video_likes","video_duration","video_category_id","video_tags"]
].reset_index(drop=True)

# ======================
# DIM EVENT
# ======================
dim_event = (
    df[["evento","evento_fecha","relacion_evento","sub_tipo_evento","tipo_evento"]]
      .dropna(subset=["evento"])
      .drop_duplicates()
      .sort_values("evento_fecha")
      .reset_index(drop=True)
)

# ======================
# DIM DATE (continua, TZ local + buffer)
# ======================
# Fechas en TZ local para evitar desfasajes por medianoche
_comment_local = df["comment_time"].dt.tz_convert(LOCAL_TZ)
_video_local   = df["video_published_at"].dt.tz_convert(LOCAL_TZ)
_evento_local  = df["evento_fecha"].dt.tz_convert(LOCAL_TZ) if "evento_fecha" in df.columns else pd.Series([], dtype="datetime64[ns, UTC]")

fechas_all = pd.concat([_comment_local, _video_local, _evento_local], ignore_index=True).dropna()
if fechas_all.empty:
    min_dt_local = pd.Timestamp("2024-01-01").normalize()
    max_dt_local = pd.Timestamp.today(tz=LOCAL_TZ).normalize().tz_localize(None)
else:
    min_dt_local = fechas_all.min().normalize().tz_localize(None)
    max_dt_local = fechas_all.max().normalize().tz_localize(None)

# Buffer para rolling
min_dt_local = min_dt_local - pd.Timedelta(days=BUFFER_DAYS)
max_dt_local = max_dt_local + pd.Timedelta(days=BUFFER_DAYS)

# Rango continuo diario (naive date)
dates = pd.date_range(
    start=min_dt_local.tz_localize(LOCAL_TZ),
    end=max_dt_local.tz_localize(LOCAL_TZ),
    freq="D",
    name="date"
).tz_convert(LOCAL_TZ).tz_localize(None)

dim_date = pd.DataFrame({"date": dates})
dim_date["year"]        = dim_date["date"].dt.year
dim_date["month"]       = dim_date["date"].dt.month
dim_date["day"]         = dim_date["date"].dt.day
dim_date["quarter"]     = dim_date["date"].dt.quarter
dim_date["ym"]          = dim_date["date"].dt.strftime("%Y-%m")
dim_date["dow"]         = dim_date["date"].dt.dayofweek              # 0=Lun
iso = dim_date["date"].astype("datetime64[ns]").dt.isocalendar()
dim_date["week_iso"]    = iso.week.astype(int)
dim_date["year_iso"]    = iso.year.astype(int)
dim_date["ym_sort"]     = dim_date["date"].dt.strftime("%Y%m").astype(int)
dim_date["date_id"]     = dim_date["date"].dt.strftime("%Y%m%d").astype(int)
dim_date["month_name"]  = dim_date["date"].dt.strftime("%b")         # Ene, Feb...
dim_date["dow_name"]    = dim_date["date"].dt.strftime("%a")         # Lun, Mar...
dim_date["is_month_start"] = dim_date["date"].dt.is_month_start
dim_date["is_month_end"]   = dim_date["date"].dt.is_month_end
dim_date["is_weekend"]     = dim_date["dow"].isin([5,6])

# ======================
# FACT COMMENTS (grano = comment_id)
# ======================
fact_cols = [
    "comment_id",
    # FKs
    "user_id","channel_id","_video_key","evento",
    # time / measures
    "comment_time","comment_likes","total_reply_count","is_top_level_comment",
    # labels / text flags
    "label_final","insulto","n_insultos","lang","term_especial","eje_argumentativo",
    # trazas útiles
    "video_title","video_published_at","channel_title","condiciones_cuenta",
    # user status
    "user_segment","bot_flag","bot_score"
]
fact_comments = df[fact_cols].rename(columns={"_video_key":"video_key"}).reset_index(drop=True)

# 🔗 Clave de relación con dim_date: día local (naive) derivado de comment_time
fact_comments["comment_date"] = _to_local_floor_day(fact_comments["comment_time"], LOCAL_TZ)

# ======================
# Escritura CSVs
# ======================
OUT_DIR.mkdir(parents=True, exist_ok=True)
dim_user.to_csv(OUT_DIR/"dim_user.csv", index=False, encoding="utf-8-sig")
dim_channel.to_csv(OUT_DIR/"dim_channel.csv", index=False, encoding="utf-8-sig")
dim_video.to_csv(OUT_DIR/"dim_video.csv", index=False, encoding="utf-8-sig")
dim_event.to_csv(OUT_DIR/"dim_event.csv", index=False, encoding="utf-8-sig")
dim_date.to_csv(OUT_DIR/"dim_date.csv", index=False, encoding="utf-8-sig")
fact_comments.to_csv(OUT_DIR/"fact_comments.csv", index=False, encoding="utf-8-sig")

# Copia opcional de n-gramas si existen
proc_dir = Path("../data/processed")
for name in ["pbi_unigrams.csv","pbi_bigrams.csv"]:
    src = proc_dir/name
    if src.exists():
        pd.read_csv(src).to_csv(OUT_DIR/name, index=False, encoding="utf-8-sig")

# Limpieza auxiliar del master
df.drop(columns=["_video_key"], inplace=True, errors="ignore")

# Resumen
print("=== BI layer escrita en:", OUT_DIR.resolve(), "===")
for name in ["dim_user","dim_channel","dim_video","dim_event","dim_date","fact_comments","pbi_unigrams","pbi_bigrams"]:
    p = OUT_DIR/f"{name}.csv"
    if p.exists():
        try:
            n = sum(1 for _ in open(p, "r", encoding="utf-8-sig")) - 1
            print(f"  - {name}.csv: ~{n} filas")
        except Exception:
            pass


In [ ]:
# ======================
# FACT COMMENTS AUX — solo NUEVAS columnas + comentario
# ======================

# Columnas del fact (no repetir)
fact_cols = [
    "comment_id","user_id","channel_id","_video_key","evento",
    "comment_time","comment_likes","total_reply_count","is_top_level_comment",
    "label_final","insulto","n_insultos","lang","term_especial","eje_argumentativo",
    "video_title","video_published_at","channel_title","condiciones_cuenta",
    "user_segment","bot_flag","bot_score","comment_date"
]

# 1) Lista explícita 
#    (comentario y outputs del modelo / features útiles)
wanted_aux = [
    "comment",                 # <-- texto original (sí incluir)
    "label_rule","regla_aplicada",
    "label_ml","ml_proba_max","ml_margen","ml_entropia",
    "clasificacion_origen",
    "user_rank","user_n_comments","user_days_active","user_freq_diaria",
    "user_country","user_region",
]

# 2) Exclusiones estrictas
EXCLUDE = {
    "comment_clean", "text_with_ctx",  # no incluir
    "bot_flag", "bot_score",           # ya están en fact
}

# 3) Armar columnas finales:
#    - tomar las wanted que EXISTAN en df y NO estén en fact ni en EXCLUDE
aux_cols = [c for c in wanted_aux if c in df.columns and c not in fact_cols and c not in EXCLUDE]

# (Opcional) auto-incluir cualquier columna NUEVA futura que no choque con fact ni EXCLUDE
AUTO_INCLUDE_NEW = True
if AUTO_INCLUDE_NEW:
    extras = [
        c for c in df.columns
        if c not in set(fact_cols) | EXCLUDE | {"_video_key"}   # evita técnicas
           and c not in aux_cols
           and c != "comment_id"
    ]
    aux_cols += extras

# Asegurar siempre comment_id
base_cols = ["comment_id"]

# Construcción del AUX
fact_comments_aux = (
    df[base_cols + [c for c in aux_cols if c in df.columns]]
      .drop_duplicates(subset=["comment_id"], keep="last")
      .reset_index(drop=True)
)

# Tipado: IDs a string; métricas a numérico cuando aplique
fact_comments_aux["comment_id"] = fact_comments_aux["comment_id"].astype("string")
for c in fact_comments_aux.columns:
    if c == "comment_id":
        continue
    if c.startswith(("ml_", "proba_", "n_", "user_", "comment_text_length")):
        fact_comments_aux[c] = pd.to_numeric(fact_comments_aux[c], errors="ignore")
    elif fact_comments_aux[c].dtype == "object":
        fact_comments_aux[c] = fact_comments_aux[c].astype("string")

# Validación de cobertura vs fact
try:
    fact_ids = set(fact_comments["comment_id"].astype("string").dropna().unique())
    aux_ids  = set(fact_comments_aux["comment_id"].astype("string").dropna().unique())
    huérfanos = aux_ids - fact_ids
    if huérfanos:
        print(f"{len(huérfanos)} comment_id en AUX no existen en fact_comments (no matchearán).")
except Exception:
    pass

# Escritura
(OUT_DIR / "fact_comments_aux.csv").parent.mkdir(parents=True, exist_ok=True)
fact_comments_aux.to_csv(OUT_DIR/"fact_comments_aux.csv", index=False, encoding="utf-8-sig")
print("  - fact_comments_aux.csv:", len(fact_comments_aux), "filas")


C:\Users\User\AppData\Local\Temp\ipykernel_14848\3859188844.py:62: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  fact_comments_aux[c] = pd.to_numeric(fact_comments_aux[c], errors="ignore")


  - fact_comments_aux.csv: 113583 filas
